#############################################################################################################
#### Copyright [2022] Capgemini.com, Inc. or its affiliates. All Rights Reserved.                           #
### Licensed under 'IDEA by Capgemini' License. You may not use this file except in compliance with the License. #
#############################################################################################################

In [1]:
# Importing the Keras libraries and packages
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model

2022-09-11 08:08:41.821731: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-11 08:08:41.821774: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


### Model Building

In [2]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a third convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))


2022-09-11 08:13:46.360918: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-11 08:13:46.361489: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-11 08:13:46.361561: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-11 08:13:46.361629: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vertex-mlops-dev-notebook): /proc/driver/nvidia/version does not exist
2022-09-11 08:13:46.363405: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


### Model Compiling

In [3]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Image loading and Preprocessing

In [13]:
import os
import shutil

os.listdir("data/test")

['normal', 'faulty']

In [12]:
shutil.rmtree("data/test/.ipynb_checkpoints")

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('data/train',target_size = (64, 64),batch_size = 32,class_mode = 'binary')
test_set = test_datagen.flow_from_directory('data/test',target_size = (64, 64),batch_size = 32, class_mode = 'binary')

Found 240 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


### Training and Saving

In [19]:
import math

BATCH_SIZE=32

TRAINING_SIZE = 240

VALIDATION_SIZE = 32

# We take the ceiling because we do not drop the remainder of the batch
compute_steps_per_epoch = lambda x: int(math.ceil(1. * x / BATCH_SIZE))

steps_per_epoch = compute_steps_per_epoch(TRAINING_SIZE)
val_steps = compute_steps_per_epoch(VALIDATION_SIZE)

print(steps_per_epoch)
print(val_steps)

8
1


In [20]:
classifier.fit(training_set,
               steps_per_epoch = 8,
               epochs = 100,
               validation_data = test_set,
               validation_steps = 1)
classifier.save('SolarPanelFaultDetection.h5')

Epoch 1/100
8/8 [==============================] - 1s 124ms/step - loss: 0.6958 - accuracy: 0.4833 - val_loss: 0.6787 - val_accuracy: 0.5333
Epoch 2/100
8/8 [==============================] - 1s 117ms/step - loss: 0.6755 - accuracy: 0.5833 - val_loss: 0.6816 - val_accuracy: 0.5333
Epoch 3/100
8/8 [==============================] - 1s 112ms/step - loss: 0.6730 - accuracy: 0.5375 - val_loss: 0.6803 - val_accuracy: 0.5000
Epoch 4/100
8/8 [==============================] - 1s 124ms/step - loss: 0.6709 - accuracy: 0.5500 - val_loss: 0.6773 - val_accuracy: 0.5333
Epoch 5/100
8/8 [==============================] - 1s 115ms/step - loss: 0.6650 - accuracy: 0.5792 - val_loss: 0.6740 - val_accuracy: 0.5667
Epoch 6/100
8/8 [==============================] - 1s 117ms/step - loss: 0.6642 - accuracy: 0.5625 - val_loss: 0.6689 - val_accuracy: 0.5667
Epoch 7/100
8/8 [==============================] - 1s 115ms/step - loss: 0.6676 - accuracy: 0.5792 - val_loss: 0.6644 - val_accuracy: 0.5333
Epoch 8/100
8